encoder transformer层的linear1层（etl1）替换为近似矩阵乘法

In [1]:
import numpy as np
import os
import sys
dir_now = os.getcwd()
sys.path.append(dir_now)
sys.path.append(os.path.join(dir_now, '../'))
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # 防止jupyter爆内存
import matmul as mm
import math_util as mu
from NNutils import *
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
method = METHOD_MITHRAL
quantize_lut = False
# method = METHOD_PQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE
nbits = 4

In [3]:
linear_name = "etl1"
linear_name_full = "ex_linear1"
feedback_bits = 256
ncodebooks = 64 # max:64
ncentroids = 256
# if method == METHOD_MITHRAL:
#     ncentroids = 16
train_sam_num = 3000 # 训练集样本数
test_sam_num = 1000 # 测试集样本数
batch_size = 32
if method == METHOD_EXACT:
    ncodebooks = 0
    ncentroids = 0

In [4]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    linearin_path_train= ''
    linearout_path_train= ''
    linearin_path_test = ''
    linearout_path_test = ''
elif host_name == 'jm-System-Product-Name':
    dir_joined = '/data/hdr/transformer_data/joined'
    dir_train = os.path.join(dir_joined, 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join(dir_joined, 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    linearin_path_train= '%sin_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
    y_train = '%s_y_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
    linearout_path_train= '%sout_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
    linearin_path_test = '%sin_test_f%i_sam%i.npy' % (linear_name_full, feedback_bits, test_sam_num)
    linearout_path_test = '%sout_test_f%i_sam%i.npy' % (linear_name_full, feedback_bits, test_sam_num)
else:
    raise NameError("You are running the script in a new computer %s, please define dirs" % host_name)


weightpath = '%s_w_f%i.npy' % (linear_name_full, feedback_bits)
biaspath = '%s_b_f%i.npy' % (linear_name_full, feedback_bits)
dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, linear_name)
try:
    os.mkdir(dir_result)
except FileNotFoundError:
    os.makedirs(dir_result)
except FileExistsError:
    pass 


In [5]:
dataset_prepare(dir_joined, linear_name_full, feedback_bits, [train_sam_num, test_sam_num], batch_size, S1 = S1)

In [6]:
est3 = mm.estFactory(X_path=linearin_path_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut)


running method:  Mithral
X.shape:  (3072000, 64)
_learn_mithral_initialization heuristic pq
learn_multisplits(): initial loss:    25436.17445634681
learn_multisplits(): returning loss:  1.88354576051834
learn_multisplits(): initial loss:    15694.775657208442
learn_multisplits(): returning loss:  1.283456442761235
learn_multisplits(): initial loss:    15728.0521482083
learn_multisplits(): returning loss:  1.0969937080517411
learn_multisplits(): initial loss:    22308.88179487992
learn_multisplits(): returning loss:  589.9522045374089
learn_multisplits(): initial loss:    22831.236562942184
learn_multisplits(): returning loss:  90.77724133604136
learn_multisplits(): initial loss:    16794.41131759254
learn_multisplits(): returning loss:  2.8455731101208865
learn_multisplits(): initial loss:    29087.724192887636
learn_multisplits(): returning loss:  3.37334091130304
learn_multisplits(): initial loss:    50927.79927459685
learn_multisplits(): returning loss:  8.742050090332924
learn_mult

In [ ]:
x_test = np.load(dir_test+'/'+linearin_path_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [ ]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1]) #AMM字典模式需要复原y大小
print("y_out_last_re.shape: ", y_out_last_re.shape)
if method == METHOD_EXACT:
    train_sam_num = 0 # 训练集样本数
if method == METHOD_SCALAR_QUANTIZE:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_nbits%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, nbits)), y_out_last_re.astype(np.float32))
elif method == METHOD_MITHRAL:
    np.save(os.path.join(dir_result, '%s%s_ql%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % (method, linear_name, quantize_lut, train_sam_num, test_sam_num, feedback_bits, ncodebooks, ncentroids)), y_out_last_re)
else:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, ncodebooks, ncentroids)), y_out_last_re)